# Установка необходимых пакетов

In [ ]:
!pip install unsloth xformers trl peft accelerate bitsandbytes faiss-cpu langchain-community langchain-huggingface sentence-transformers PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64

In [ ]:
import os
import gdown
import json
import torch
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

adapters_name = "llama_3_8b_full"
index_name = "russco_faiss"

adapters_id = "1c9N3vpGaqt7_HOIXnErj18NdJ9XCcW2i"
index_id = "18wRauo11wkQlPT6gMmy-6AlMLHWt77sG"

# Скачивание и распаковка адаптеров
gdown.download(id=adapters_id, output="adapters.zip", quiet=False)
!unzip -o -q adapters.zip -d adapters
print("Adapters downloaded and unzipped.")

# Скачивание и распаковка базы FAISS
gdown.download(id=index_id, output="index.zip", quiet=False)
!unzip -o -q index.zip -d index
print("FAISS index downloaded and unzipped.")

# Загрузка основной модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "adapters/"+adapters_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# Загрузка модели эмбеддингов
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Загрузка базы данных FAISS
db = FAISS.load_local(
    "index/"+index_name,
    embeddings,
    allow_dangerous_deserialization=True
)
links_path = "index/"+index_name+"/links.json"
with open(links_path, 'r', encoding='utf-8') as file:
    links_dict = json.load(file)


def get_context(treatment_plan):
    search_query = f"query: {treatment_plan}"
    docs = db.similarity_search(search_query, k=2)

    context = "\n---\n".join([
        f"Источник ({d.metadata.get('Header 2', 'Общее')}): {d.page_content}"
        for d in docs
    ])
    sources = [d.metadata.get('source', 'RUSSCO') for d in docs]
    return context, sources

prompt_template = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.
{}
### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
"""

def correct_treatment_plan(treatment_plan, context, doctor=True):
    added_text = "Формулируй ответ простыми словами, которые будут понятны пациенту.\n"
    if doctor:
        added_text = ""
    final_prompt = prompt_template.format(added_text, treatment_plan, context)
    inputs = tokenizer([final_prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    result = tokenizer.batch_decode(outputs, skip_special_tokens = True)
    corrections = result[0][len(final_prompt):]
    return corrections

In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
from google.colab import files
import PyPDF2
import io
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import getpass

class MedicalAssistant:

    def __init__(self):
        self.setup_ui()

    def setup_ui(self):
        title = widgets.HTML(
            value="<h1 style='color: #2c3e50; text-align: center;'>🏥 Медицинский AI ассистент</h1>"
        )

        self.role = widgets.Dropdown(
            options=[('Пациент', 'patient'), ('Врач', 'doctor')],
            value='patient',
            description='Вы:',
            style={'description_width': 'initial'}
        )

        tab = widgets.Tab()

        self.patient_info = widgets.Textarea(
            value='',
            placeholder='Введите информацию о пациенте...',
            description='Информация:',
            layout=widgets.Layout(width='100%', height='150px')
        )

        self.upload_button = widgets.Button(
            description='Загрузить PDF',
            button_style='primary',
            icon='upload'
        )
        self.upload_button.on_click(self.upload_pdf)

        self.pdf_output = widgets.Output()

        self.submit_patient_info = widgets.Button(
            description='Отправить информацию',
            button_style='success',
            icon='check'
        )
        self.submit_patient_info.on_click(self.process_patient_info)

        patient_tab = widgets.VBox([
            widgets.HTML("<h3>📋 Информация о пациенте</h3>"),
            self.role,
            self.patient_info,
            widgets.HBox([self.upload_button]),
            self.pdf_output,
            self.submit_patient_info
        ])

        self.support_email = widgets.Text(
            value='',
            placeholder='Ваш email для ответа',
            description='Ваш email:',
            layout=widgets.Layout(width='100%')
        )

        self.support_message = widgets.Textarea(
            value='',
            placeholder='Опишите вашу проблему или вопрос...',
            description='Сообщение:',
            layout=widgets.Layout(width='100%', height='150px')
        )

        self.support_subject = widgets.Text(
            value='',
            placeholder='Тема сообщения',
            description='Тема:',
            layout=widgets.Layout(width='100%')
        )

        self.send_support = widgets.Button(
            description='Отправить в техподдержку',
            button_style='warning',
            icon='envelope'
        )
        self.send_support.on_click(self.send_support_email)

        self.support_output = widgets.Output()

        support_tab = widgets.VBox([
            widgets.HTML("<h3>📧 Техническая поддержка</h3>"),
            self.support_email,
            self.support_subject,
            self.support_message,
            self.send_support,
            self.support_output
        ])

        self.results_output = widgets.Output()

        results_tab = widgets.VBox([
            widgets.HTML("<h3>📊 Результаты обработки</h3>"),
            self.results_output
        ])

        tab.children = [patient_tab, support_tab, results_tab]
        tab.set_title(0, '👤 Пациент')
        tab.set_title(1, '🆘 Поддержка')
        tab.set_title(2, '📊 Результаты')

        display(widgets.VBox([title, tab]))

    def upload_pdf(self, b):
        with self.pdf_output:
            clear_output(wait=True)
            uploaded = files.upload()

            for filename in uploaded.keys():
                pdf_content = uploaded[filename]

                try:
                    pdf_file = io.BytesIO(pdf_content)
                    pdf_reader = PyPDF2.PdfReader(pdf_file)

                    text = ""
                    for page in pdf_reader.pages:
                        text += page.extract_text()

                    current_text = self.patient_info.value
                    self.patient_info.value = current_text + "\n\n[Извлечено из PDF:]\n" + text[:500] + "..."

                    print(f"✅ Файл {filename} успешно загружен и обработан")
                    print(f"📄 Извлечено {len(pdf_reader.pages)} страниц")

                except Exception as e:
                    print(f"❌ Ошибка при обработке PDF: {str(e)}")

    def process_patient_info(self, b):
        with self.results_output:
            clear_output(wait=True)

            role_text = "Врач" if self.role.value == "doctor" else "Пациент"
            doctor = True if self.role.value == "doctor" else False
            info = self.patient_info.value

            if not info.strip():
                print("⚠️ Пожалуйста, введите информацию о пациенте")
                return

            print(f"👤 Роль: {role_text}")
            print(f"📝 Полученная информация:\n{info}\n")
            print("🤖 AI ассистент обрабатывает запрос...")

            context, sources = get_context(treatment_plan)
            corrections = correct_treatment_plan(treatment_plan, context, doctor)

            print(f"🤖 Ответ AI ассистента:\n{corrections}")
            print("\n📖 Источники:")
            link_list = []
            for source in sources:
                source_link = links_dict[source]
                link = widgets.HTML(f"<a href='{source_link}' target='_blank'>{source}</a>")
                link_list.append(link)
            display(widgets.VBox(link_list))

    def send_support_email(self, b):
        with self.support_output:
            clear_output(wait=True)

            SMTP_SERVER = "smtp.gmail.com"
            SMTP_PORT = 587
            SENDER_EMAIL = "your_email@gmail.com"
            SENDER_PASSWORD = "your_password"

            to_email = "support@medicalassistant.com"  # чья-то почта
            subject = self.support_subject.value or "Обращение в техподдержку"
            message = self.support_message.value
            user_email = self.support_email.value

            if not message:
                print("⚠️ Пожалуйста, введите сообщение")
                return

            try:
                msg = MIMEMultipart()
                msg['From'] = SENDER_EMAIL
                msg['To'] = to_email
                msg['Subject'] = f"[Техподдержка] {subject}"

                body = f"От: {user_email}\n\n{message}"
                msg.attach(MIMEText(body, 'plain'))


                print("✅ Сообщение отправлено в техподдержку")
                print("📧 Мы свяжемся с вами в ближайшее время")

                self.support_message.value = ""
                self.support_subject.value = ""

            except Exception as e:
                print(f"❌ Ошибка при отправке: {str(e)}")
                print("\n💡 Для работы функции отправки email необходимо:")
                print("1. Включить двухфакторную аутентификацию в Google")
                print("2. Создать пароль приложения")
                print("3. Заменить SENDER_EMAIL и SENDER_PASSWORD в коде")

# Запуск модели

In [13]:
assistant = MedicalAssistant()

# OLD

Диагноз Немелкоклеточный рак легкого (НМРЛ)   
Анализы: EGFR, ALK, ROS1, BRAF, PD-L1 (TPS), RET, MET, KRAS G12C, NTRK, HER2.
назначение:  
Адъювантно: Осимертиниб (EGFR+), Алектиниб (ALK+), Атезолизумаб (PD-L1≥50% после ХТ).  
Неоадъвантно (III стадия): ХТ + Ниволумаб/Пембролизумаб/Дурвалумаб. ХТ на основе платины (Этопозид + Цисплатин)  
Метастатический 1-я линия: При драйверах — Таргетная терапия (Осимертиниб, Алектиниб и др.).
При PD-L1≥50% — моноиммунотерапия (Пембролизумаб). При PD-L1<50% — ХТ + иммунотерапия.

In [7]:
treatment_plan = """Диагноз: рак молочной железы.
Для определения тактики лечения проводится комплексное исследование опухоли:
ИГХ-профиль: Оценка рецепторов к гормонам (эстрогену и прогестерону), статуса HER2 и индекса пролиферации Ki-67.
Генетическое тестирование:
Поиск мутаций BRCA1/2 (обязательно при семейной истории болезни или специфических формах рака).
Анализ PIK3CA (необходим при метастатическом гормонозависимом раке для подбора таргетной терапии).
Определение уровня PD-L1 (критично для выбора иммунотерапии при тройном негативном подтипе).
Алгоритмы медикаментозного лечения
1. Профилактическое (адъювантное) лечение:
HER2-положительный тип: Стандарт — трастузумаб в течение года. При поражении лимфоузлов добавляется пертузумаб. Если после предоперационной химии осталась опухоль, назначается Кадсила (T-DM1).
Тройной негативный тип: При остаточной опухоли применяется капецитабин. Пациентам с мутацией BRCA показан олапариб.
2. Терапия при распространенных (метастатических) формах:
Гормонозависимый HER2-отрицательный: Основной стандарт первой линии — комбинация ингибиторов CDK4/6 (рибо-, палбо- или абемациклиб) с гормональной терапией. При наличии мутации PIK3CA к фулвестранту добавляют алпелисиб.
HER2-положительный: Начинают с комбинации таксанов и двойной блокады (трастузумаб + пертузумаб). Во второй линии используют T-DM1, в последующих — инновационный конъюгат T-DXd.
Тройной негативный: При высоком уровне PD-L1 эффективно сочетание химиотерапии с иммунопрепаратом пембролизумабом. Носителям мутации BRCA назначаются таргетные PARP-ингибиторы (олапариб/талазопариб).
"""

context, sources = get_context(treatment_plan)
corrections = correct_treatment_plan(treatment_plan, context)
print(corrections)

Ошибка в плане лечения: При адъювантном лечении рака поджелудочной железы не рекомендуется назначение трастузумаба. Схема терапии при этом типе опухоли должна основываться на химиотерапии и/или эндокринной терапии.


In [9]:
sources

['нарушения_когнитивных_функций', 'рак_молочной_железы']